# Quickstart LSST Verify Demo

<br>Owner: **Keith Bechtol** ([@bechtol](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@bechtol))
<br>Last Verified to Run: **d_2019_02_04**
<br>Verified Stack Release: **d_2019_02_04**, **d_2019_01_15**, need to check v16.0

LSST Verify python pakcage: https://github.com/lsst/verify

This notebook is based on the documentation at https://sqr-019.lsst.io/

Another example: https://github.com/mareuter/notebooks/blob/master/LSST/Systems_Engineering/System_Verification_SQuaSH/System_Verification_Demo.ipynb

for which the metrics are defined here:
https://github.com/mareuter/notebooks/tree/master/LSST/Systems_Engineering/System_Verification_SQuaSH

## Setup

In [ ]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
import json
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import astropy.units as u

import lsst.verify

## Defining metrics and specifications

Rules for directory structure and naming

In [ ]:
!tree verify_demo

Let's take a look at the files

In [ ]:
!cat verify_demo/metrics/demo.yaml

In [ ]:
!cat verify_demo/specs/demo/specs.yaml

Note that the "---" between specifications is required

In [ ]:
METRIC_PACKAGE = "verify_demo"
metrics = lsst.verify.MetricSet.load_metrics_package(METRIC_PACKAGE)
specs = lsst.verify.SpecificationSet.load_metrics_package(METRIC_PACKAGE)

In [ ]:
metrics

In [ ]:
specs

## Computing and storing metrics

These are completely made up values for purpose of illustration

In [ ]:
zp_rms = 15.*u.mmag
zp_meas = lsst.verify.Measurement('demo_photometry.ZeropointRMS', zp_rms)

astro_rms = 15.*u.mas
astro_meas = lsst.verify.Measurement('demo_astrometry.AstrometricRMS', astro_rms)

It is possible to include extra information (again, completely made up values)

In [ ]:
zp_meas.extras['x'] = lsst.verify.Datum(np.random.random(10) * u.mag, label="x", description="x-values")
zp_meas.extras['y'] = lsst.verify.Datum(np.random.random(10) * u.mag, label="y", description="y-values")

Create an LSST verify job and add the measurements

In [ ]:
job = lsst.verify.Job.load_metrics_package()
job.metrics.update(metrics)
job.specs.update(specs)
job.measurements.insert(zp_meas)
job.measurements.insert(astro_meas)

Provide metadata

In [ ]:
job.meta.update({'version': 'test'})

Write the output to a file

In [ ]:
job.write('demo.json')

## Creating reports

Create a report. Read the file

In [ ]:
with open('demo.json') as f:
    job = lsst.verify.Job.deserialize(**json.load(f))

In [ ]:
job.report().show()

Select on particular tags

In [ ]:
job.report(spec_tags=['minimum']).show()

In [ ]:
#job.report(metric_tags=['demo']).show()
job.metrics['demo_astrometry.AstrometricRMS'].tags

In [ ]:
#job.report(metric_tags=['minimum']).show()

In [ ]:
job.meta

In [ ]:
m = job.measurements['demo_photometry.ZeropointRMS']

A lot of information is available for plotting 

In [ ]:
plt.figure()
plt.scatter(m.extras['x'].quantity, m.extras['y'].quantity)
plt.xlabel('%s (%s)'%(m.extras['x'].label, m.extras['x'].unit.name))
plt.ylabel('%s (%s)'%(m.extras['y'].label, m.extras['y'].unit.name))
plt.title('%s; %s'%(m.metric_name.metric, job.meta["version"]))
plt.xlim(0, 1)
plt.ylim(0, 1)